In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
# import torch

In [2]:
subject_common = pd.read_csv('data/subject_common_info.csv')
subject_common['subject_type'] = "common"
subject_common['subject_grade'] = 1
select_common = ["화법과 작문","독서","언어와 매체","문학","수학Ⅰ","수학Ⅱ","미적분","확률과 통계","영어회화","영어Ⅰ","영어독해와 작문","영어Ⅱ","한국지리","세계지리","세계사","동아시아사","경제","정치와 법","사회·문화","생활과 윤리","윤리와 사상","물리학Ⅰ","화학Ⅰ","생명과학Ⅰ","지구과학Ⅰ","체육","운동과 건강","음악","미술","연극","기술·가정","정보","독일어Ⅰ","한문Ⅰ","철학","논리학","심리학","교육학","종교학","진로와 직업","보건","환경","실용 경제","논술"]

#check text is in select_common
def check_common(text):
    return any(i in text for i in select_common)

#선택교과
subject_names = []
subject_desc = []
subject_type = []
subject_grade = []

for i in range(1,100):
    page = requests.get(f"https://www.cbe.go.kr/dept/19/sub.php?menukey=4496&mod=list1&code1=1&code2={i}")
    soup = bs(page.text, "html.parser")
    soup = soup.find("div",id="contentsType")
    try:
        subject_name = soup.find("h3").text
        subject_names.append(subject_name)
    except Exception:
        continue
    elements = soup.find_all('p')
    subject_desc.append(elements[0].text) # 과목설명
    if check_common(subject_name):
        subject_type.append("select_common")
        subject_grade.append(2)
    else:
        subject_type.append("select_career")
        subject_grade.append(3)
subject_select = pd.DataFrame({"name":subject_names,"description":subject_desc, "subject_type":subject_type, "subject_grade":subject_grade})


In [7]:
def crawler_prof(menu_key, code1, start, end):
    for i in range(start,end):
        page = requests.get(f"https://www.cbe.go.kr/dept/19/sub.php?menukey={menu_key}&mod=list2&code1={code1}&code2={i}")
        soup = bs(page.text, "html.parser")
        soup = soup.find("div",id="contentsType")
        try:
            subject_name = soup.find("h3").text
            subject_names.append(subject_name)
        except Exception:
            print(i)
            continue
        elements = soup.find_all('p')
        # print(elements)
        try:
            subject_desc.append(elements[0].text) # 과목설명
        except Exception:
            elements = soup.find_all('li')
            desc = elements[0].text[3:]+elements[1].text[3:]+elements[2].text[3:]
            subject_desc.append(desc) # 과목설명(li 가나다)


In [8]:
#전문교과
subject_names = []
subject_desc = []
subject_grade = []

crawler_prof(4538, 97, 98, 114)
crawler_prof(4538, 114, 115, 129)
crawler_prof(4538, 129, 130, 178)
crawler_prof(4538, 179, 180, 235)
crawler_prof(4538, 236, 237, 248)

subject_type = ["professional_1" for _ in range(len(subject_names))]
subject_grade = [3 for _ in range(len(subject_names))]
subject_professional = pd.DataFrame({"name":subject_names,"description":subject_desc, "subject_type":subject_type,"subject_grade":subject_grade})


In [9]:
subjects = pd.concat([subject_common, subject_select, subject_professional],axis=0)
subjects.to_csv('data/subject_info.csv',header=True)